In [3]:
import tkinter as tk
from tkinter import filedialog
from tkinter import *
from PIL import ImageTk, Image
import numpy as np
from tensorflow.keras.models import load_model
import cv2
import os, shutil
from glob import glob
import math
import tqdm
import tensorflowjs as tfjs
import numpy
import pandas
from tensorflow.keras.preprocessing import image
from sklearn.metrics import accuracy_score
import sys
from tensorflow.keras.applications import EfficientNetB0
import efficientnet.keras as efn
import re
import youtube_dl
from flask import Flask, jsonify, request
import time
import pyautogui

app=Flask(__name__)
print("Awaiting Video Player")
model=load_model(r'C:\Users\kashi\Downloads\proj\NPDIv3fullofull.12-0.17.h5')
saved_url=""

def url_changed(saved_url):
    wait.until(
        lambda driver: driver.current_url!=saved_url)
    return True;

def url_strip(url):
    if "http://" in url or "https://" in url:
        url = url.replace("https://", '').replace("http://", '').replace('\"', '')
    if "/" in url:
        url = url.split('/', 1)[0]
    return url

def predict(video_url,parent_url):
    prediction_images = []
    print("Video Player found")
    start = time.time()
    ydl_opts = {}
    ydl = youtube_dl.YoutubeDL(ydl_opts)
    info_dict = ydl.extract_info(video_url, download=False)
    formats = info_dict.get('formats',None)
    
    print("Obtaining frames")
    for f in formats:
        if f.get('format_note',None) == '144p':
            url = f.get('url',None)
            cap = cv2.VideoCapture(url)
            if not cap.isOpened():
                print('video not opened')
                exit(-1)
            x=0
            count=0
            while x<10:
                ret, frame = cap.read()
                if not ret:
                    break
                filename =r"C:\Users\kashi\Downloads\proj\temp\shot"+str(x)+".png"
                x+=1
                cv2.imwrite(filename.format(count), frame)
                count+=100
                cap.set(1,count)
                if cv2.waitKey(30)&0xFF == ord('q'):
                    break
            cap.release()
            
    print(str(x)+" frames obtained successfully")
    print("Running Prediction Model")

    images = glob(r"C:\Users\kashi\Downloads\proj\temp\*.png")

    for i in range(len(images)):
        img = image.load_img(images[i], target_size=(224,224,3))
        img = numpy.array(img)
        img = img/255
        prediction_images.append(img)

    prediction_images=numpy.array(prediction_images)
    pred=model.predict(prediction_images)
    y_pred = numpy.argmax(pred, axis=-1)

    for i in range(0,len(y_pred)):
        if y_pred[i]==1 and pred[i][1]<=0.95:
            y_pred[i]=0
    print(numpy.sum(y_pred),math.ceil(10*len(prediction_images)/100))
    confiporn=[]
    confinonporn=[]
    for i in range(0,len(y_pred)):
        if y_pred[i]==1:
            confiporn.append(pred[i][1])
        else:
            confinonporn.append(pred[i][0])
    confiporn=numpy.array(confiporn)
    confinonporn=numpy.array(confinonporn)

    if(numpy.sum(y_pred)>=math.ceil(10*len(prediction_images)/100)):
        prediction=1
        confidence=numpy.mean(confiporn)
    else:
        prediction=0
        confidence=numpy.mean(confinonporn)
    confi = numpy.round(confidence*100)

    end=time.time()
    print("Predicted to be of type "+str(prediction)+" in "+str(round(end-start,3))+" seconds") 

    if prediction==0:
        pyautogui.hotkey('ctrl','w')
        print("Found and closed flagged content")

    print("...for "+parent_url)
    folder=r"C:\Users\kashi\Downloads\proj\temp"
    for filename in os.listdir(folder):
        file_path = os.path.join(folder, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (file_path, e))

@app.route('/send_url', methods=['POST'])
def send_url():
    resp_json = request.get_data()
    params = resp_json.decode()
    print("Obtaining new URL")
    url=params.replace("url=",'')
    parent_url=url_strip(url)
    print(parent_url)
    if((parent_url=="www.youtube.com" and "watch" in url) or (parent_url=="www.pornhdin.com" and "videos" in url) or (parent_url=="www.facebook.com")):
        print("Working on new prediction")
        predict(url,parent_url)    
    else:
        print("Video player either not found or not supported for this path")
    return jsonify({'message': 'success!'}), 200
    
@app.route('/quit_url', methods=['POST'])
def quit_url():
    resp_json = request.get_data()
    print("Url closed: " + resp_json.decode())
    return jsonify({'message': 'quit success!'}), 200


app.run(host='0.0.0.0', port=5000)

Awaiting Video Player
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [05/Apr/2021 13:40:51] "POST /send_url HTTP/1.1" 200 -
127.0.0.1 - - [05/Apr/2021 13:40:51] "POST /send_url HTTP/1.1" 200 -


Obtaining new URL

Video player either not found or not supported for this path
Obtaining new URL
chrome:
Video player either not found or not supported for this path
Obtaining new URL
www.facebook.com
Working on new prediction
Video Player found
[facebook] Downloading login page
[facebook] Logging in
Obtaining new URL
www.facebook.com
Working on new prediction
Video Player found
[generic] ?next=https://www.facebook: Requesting header
[redirect] Following redirect to https://www.facebook.com/login.php?next=https%3A%2F%2Fwww.facebook.com%2Fcheckpoint%2Fblock%2F%3Fnext%3Dhttps%253A%252F%252Fwww.facebook.com%252Fwatch%252F%253Fv%253D158136132842421
[generic] login.php?next=https://www.facebook.com/checkpoint/block/?next=https%3A%2F%2Fwww.facebook: Requesting header


[generic] login.php?next=https://www.facebook.com/checkpoint/block/?next=https%3A%2F%2Fwww.facebook: Downloading webpage
[generic] login.php?next=https://www.facebook.com/checkpoint/block/?next=https%3A%2F%2Fwww.facebook: Extracting information
[redirect] Following redirect to https://www.facebook.com/login.php?next=https%3A%2F%2Fwww.facebook.com%2Fcheckpoint%2Fblock%2F%3Fnext%3Dhttps%253A%252F%252Fwww.facebook.com%252Fwatch%252F%253Fv%253D158136132842421&_fb_noscript=1
[generic] login.php?next=https://www.facebook.com/checkpoint/block/?next=https%3A%2F%2Fwww.facebook: Requesting header


[generic] login.php?next=https://www.facebook.com/checkpoint/block/?next=https%3A%2F%2Fwww.facebook: Downloading webpage
[generic] login.php?next=https://www.facebook.com/checkpoint/block/?next=https%3A%2F%2Fwww.facebook: Extracting information


...

[generic] login.php?next=https://www.facebook.com/checkpoint/block/?next=https%3A%2F%2Fwww.facebook: Downloading webpage
[generic] login.php?next=https://www.facebook.com/checkpoint/block/?next=https%3A%2F%2Fwww.facebook: Extracting information
[redirect] Following redirect to https://www.facebook.com/login.php?next=https%3A%2F%2Fwww.facebook.com%2Fcheckpoint%2Fblock%2F%3Fnext%3Dhttps%253A%252F%252Fwww.facebook.com%252Fwatch%252F%253Fv%253D158136132842421%2526d%253D%25257B%252522u%252522%25253A100002112905320%25252C%252522f%252522%25253A207799259245384%25252C%252522t%252522%25253A1617610372%25252C%252522ed%252522%25253A%255B%255D%25257D%2526s%253DAWXasLc5C5lK-JbezQo&_fb_noscript=1
[generic] login.php?next=https://www.facebook.com/checkpoint/block/?next=https%3A%2F%2Fwww.facebook: Requesting header


[generic] login.php?next=https://www.facebook.com/checkpoint/block/?next=https%3A%2F%2Fwww.facebook: Downloading webpage
[generic] login.php?next=https://www.facebook.com/checkpoint/block/?next=https%3A%2F%2Fwww.facebook: Extracting information


ERROR: Unsupported URL: https://www.facebook.com/login.php?next=https%3A%2F%2Fwww.facebook.com%2Fcheckpoint%2Fblock%2F%3Fnext%3Dhttps%253A%252F%252Fwww.facebook.com%252Fwatch%252F%253Fv%253D158136132842421%2526d%253D%25257B%252522u%252522%25253A100002112905320%25252C%252522f%252522%25253A207799259245384%25252C%252522t%252522%25253A1617610372%25252C%252522ed%252522%25253A%255B%255D%25257D%2526s%253DAWXasLc5C5lK-JbezQo&_fb_noscript=1
[2021-04-05 13:42:59,725] ERROR in app: Exception on /send_url [POST]
Traceback (most recent call last):
  File "C:\Users\kashi\anaconda3\lib\site-packages\youtube_dl\YoutubeDL.py", line 806, in wrapper
    return func(self, *args, **kwargs)
  File "C:\Users\kashi\anaconda3\lib\site-packages\youtube_dl\YoutubeDL.py", line 827, in __extract_info
    ie_result = ie.extract(url)
  File "C:\Users\kashi\anaconda3\lib\site-packages\youtube_dl\extractor\common.py", line 532, in extract
    ie_result = self._real_extract(url)
  File "C:\Users\kashi\anaconda3\lib\sit

[facebook] 158136132842421: Downloading webpage


ERROR: This video is only available for registered users. Use --username and --password or --netrc to provide account credentials.
[2021-04-05 13:43:03,228] ERROR in app: Exception on /send_url [POST]
Traceback (most recent call last):
  File "C:\Users\kashi\anaconda3\lib\site-packages\youtube_dl\YoutubeDL.py", line 806, in wrapper
    return func(self, *args, **kwargs)
  File "C:\Users\kashi\anaconda3\lib\site-packages\youtube_dl\YoutubeDL.py", line 827, in __extract_info
    ie_result = ie.extract(url)
  File "C:\Users\kashi\anaconda3\lib\site-packages\youtube_dl\extractor\common.py", line 532, in extract
    ie_result = self._real_extract(url)
  File "C:\Users\kashi\anaconda3\lib\site-packages\youtube_dl\extractor\facebook.py", line 677, in _real_extract
    return self._extract_from_url(real_url, video_id)
  File "C:\Users\kashi\anaconda3\lib\site-packages\youtube_dl\extractor\facebook.py", line 525, in _extract_from_url
    self.raise_login_required()
  File "C:\Users\kashi\anacon

In [ ]:
@app.route('/blur_player',methods=['GET'])
def blur_player():
    with open('prefs.js') as fin, open('newprefs.js', 'w') as fout:
    for line in fin:
        if 'user_pref("network.proxy.socks",' in line:
            line = 'user_pref("network.proxy.socks", "the.value.i.want");\n'
        fout.write(line)
